# Summary

I noticed several days with over 10 hours of sleep from the notebook where I obtain my sleep data via the Fitbit API. I don't remember ever sleeping more than 8 hours in one night, so what's going on?

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set_context('poster')
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
py.init_notebook_mode(connected=True)

In [4]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data', 'interim', 'sleep_data.csv')
df_sleep = pd.read_csv(data_path, index_col='shifted_datetime', parse_dates=True)

In [23]:
df_sleep.index += pd.Timedelta(hours=12)

In [24]:
sleep_day = df_sleep.resample('1D').sum().fillna(0)

In [29]:
redeyes = ['2016-12-12', '2017-01-16', '2017-07-02']

In [30]:
toplot = sleep_day['minutesAsleep']/60.

data = []

data.append(
    go.Scatter(
        x=toplot.index,
        y=toplot.values,
        name='Hours Asleep'
    )
)

shapes = []
for ired, redeye in enumerate(redeyes):
    if ired ==0:
        showlegend = True
    else:
        showlegend = False
    trace0 = go.Scatter(
        x=[redeye, redeye],
        y=[toplot.dropna().min(), toplot.dropna().max()],
        mode='lines',
        name='Redeye flight',
        line={
            'color': 'salmon',
            'width': 3,
            'dash': 'dash'
        },
        showlegend=showlegend
    )
    data.append(trace0)
    shapes.append({
        'type': 'line',
        'x0': redeye,
        'y0': 0,
        'x1': redeye,
        'y1': 1,
        'xref': 'x',
        'yref': 'paper',
        'line': {
            'color': 'salmon',
            'width': 14,
            'alpha': 0.5
        },
    })

layout = go.Layout(
    title="Daily Sleep Total, 6pm to 6pm",
    yaxis=dict(
        title='Hours Asleep'
    ),
)

fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='DailySleepTotal')

On redeye flights, I don't sleep. But, the day of arrival I get between 3.8 - 6.4 hours of sleep. So I am able to make up for part of the lost sleep.

In [45]:
print("Average daily sleep total: {:.2f} hours".format(toplot.mean()))

Average daily sleep total: 6.74 hours


In [41]:
for ired, redeye in enumerate(redeyes):
    redeye1 = pd.to_datetime(redeye) + pd.Timedelta(days=0)
    redeye3 = pd.to_datetime(redeye) + pd.Timedelta(days=2)
    redeye7 = pd.to_datetime(redeye) + pd.Timedelta(days=6)
#     print("Sleep after redeye: {:.2f} hours".format(toplot[redeye]))
    print("Average sleep per day for 1 day after redeye: {:.2f}".format(toplot[redeye:redeye1].mean()))
    print("Average sleep per day for 3 days after redeye: {:.2f}".format(toplot[redeye:redeye3].mean()))
    print("Average sleep per day for 7 days after redeye: {:.2f}".format(toplot[redeye:redeye7].mean()))

Average sleep per day for 1 day after redeye: 3.78
Average sleep per day for 3 days after redeye: 5.50
Average sleep per day for 7 days after redeye: 6.78
Average sleep per day for 1 day after redeye: 6.37
Average sleep per day for 3 days after redeye: 7.41
Average sleep per day for 7 days after redeye: 6.68
Average sleep per day for 1 day after redeye: 4.68
Average sleep per day for 3 days after redeye: 5.93
Average sleep per day for 7 days after redeye: 6.06


For two out of my three redeye flights over the past 9 months, the sleep that I lost the night of the redeye led to reduced average daily sleep totals (compared to my overall average of 6.7 hours) over the subsequent 3 days and 7 days. Both of those trips were to California to visit family. Upon return, I went back to work and did not have the luxury of sleeping in to catch up on sleep. In contrast, the other redeye flight was part of a vacation to Iceland, so I had the opportunity to sleep in a bit and catch up on sleep.

This evidence indicates that "redeye followed by work" corresponds to an expected reduction in daily sleep of 1.25 hours for the 3 days following a redeye trip and 0 - 0.7 hours for the week following a redeye trip. Redeye followed by vacation does not seem to have a significant impact on my daily sleep totals.

Based on the limited data available, it is worthwhile to include this as a significant predictive feature for my expected daily sleep total. For instance, a reasonable goal for average daily sleep over the 3 days after a redeye might be 6 hours per day rather than my typical goal of 7.3 hours.